# Helpers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 3.0 MB/s eta 0:00:00


In [ ]:
from logging import exception
import keras
import pickle
from keras_applications.resnext import ResNeXt50
from tensorflow.keras.models import Model
import tensorflow as tf
import os

from PIL import Image
from glob import glob
import numpy as np

from tqdm import tqdm

In [ ]:
def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data

# Model

In [ ]:
imageW, imageH= (224, 224)

model= ResNeXt50(
    weights="imagenet",
    backend= keras.backend,
    layers= keras.layers,
    models= keras.models,
    utils= tf.keras.utils
)


outputs=model.layers[-2].output
model_new = Model(inputs=model.inputs, outputs=model.layers[-2].output)

100853992/100853992 [==============================] - 1s 0us/step


In [ ]:
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9408        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
def process_single_image(img_file:str):
    """
    Function to convert image to vector
    """
    im= Image.open(img_file, 'r')
    im= im.resize((imageW, imageH, ))
    pixel_values= list(im.getdata())
    data= np.array(pixel_values, dtype=int).reshape(-1)
    def range_squish(x): return x/255
    data= range_squish(data)
    return data

def process_image(img_file_arr):
    all_images= []
    for img_file in img_file_arr:
        data= process_single_image(img_file)
        if len(data)!= 150528:
            print(len(data))
        if len(data) == 50176:
            data= list(data)+list(data)+list(data)
            print(len(data))
        all_images.append(data)
    all_images= np.array(all_images, dtype=np.float32)
    all_images= all_images.reshape((-1, imageH, imageW, 3))
    assert len(all_images) == len(img_file_arr)
    return all_images


def apply_resnet50(img_file_arr):
    ip= process_image(img_file_arr)
    res= model_new.predict(ip)
    mean = np.mean(res, axis=0)
    return list(mean)

In [ ]:
folder_path = "/content/drive/MyDrive/scene_detect/scene detection + object detection/"
folder_list = os.listdir(folder_path)
op = {}

In [ ]:
for i in tqdm(folder_list):

  new_single_movie = []
  single_movie = os.listdir(folder_path + i)
  for j in single_movie:
    new_id = folder_path + i + "/" + j
    new_single_movie.append(new_id)
  vec = apply_resnet50(new_single_movie)

  op[i[0:9]] = vec

  0%|          | 0/510 [00:00<?, ?it/s]

2/2 [==============================] - 13s 928ms/step


  0%|          | 1/510 [00:32<4:33:01, 32.18s/it]

3/3 [==============================] - 2s 806ms/step


  0%|          | 2/510 [01:05<4:39:42, 33.04s/it]

...

100%|█████████▉| 508/510 [1:57:12<00:11,  5.92s/it]

1/1 [==============================] - 0s 34ms/step


100%|█████████▉| 509/510 [1:57:23<00:07,  7.36s/it]

2/2 [==============================] - 0s 330ms/step


100%|██████████| 510/510 [1:57:41<00:00, 13.85s/it]


# Embedding Extraction

In [ ]:
import pickle
file_pi = open('scene_object_video_embedding.pkl', 'wb')
pickle.dump(op, file_pi)

In [ ]:
ab = load_pkl("scene_object_video_embedding.pkl")

In [ ]:
ab.keys()

dict_keys(['tt3607198', 'tt3447364', 'tt3390572', 'tt3261022', 'tt3495026', 'tt3501994', 'tt3322420', 'tt3678782', 'tt3337550', 'tt3449292', 'tt3614516', 'tt3495030', 'tt0041123', 'tt0043307', 'tt0045506', 'tt0045693', 'tt0048987', 'tt0049072', 'tt0060659', 'tt0050132', 'tt0063404', 'tt0066758', 'tt0066763', 'tt0067421', 'tt0067403', 'tt0061378', 'tt0056052', 'tt0433425', 'tt0050758', 'tt0050322', 'tt0050665', 'tt0052217', 'tt0052560', 'tt0056436', 'tt0060689', 'tt0061072', 'tt0073707', 'tt0079221', 'tt0072783', 'tt0076696', 'tt0077451', 'tt0074730', 'tt0075669', 'tt0072777', 'tt0069671', 'tt0068257', 'tt0077797', 'tt0077783', 'tt0070947', 'tt0075747', 'tt0073391', 'tt0072064', 'tt0068217', 'tt0078418', 'tt0079386', 'tt0070530', 'tt0150992', 'tt0150433', 'tt0118751', 'tt0116950', 'tt0114234', 'tt0120456', 'tt0112870', 'tt0109117', 'tt0110222', 'tt0106655', 'tt0107166', 'tt0110076', 'tt0104561', 'tt0106333', 'tt0102636', 'tt0102701', 'tt0100002', 'tt0096028', 'tt0098999', 'tt0100401', '

In [ ]:
len(ab['tt2624852'])

2048